In [1]:
import warnings

warnings.filterwarnings("ignore")

import copy
import json
import os
import re
import sys
from collections import Counter
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from rich import print
from scipy.ndimage import convolve
from tqdm import tqdm

In [2]:
def read_file_to_str_li(fp, print_exp=True):
    with open(fp, "r") as f:
        lines = f.read().split("\n")
    if print_exp:
        print(f"Read from {fp}:")
        print(f"First line: {lines[0]} | Last line: {lines[-1]}")
        print("-" * 6)

    return lines


# define the function blocks
def convert_to_int(input_str):
    if input_str == "" or input_str == " ":
        return None
    return int(input_str)


def convert_to_str(input_str):
    return str(input_str)


# map the inputs to the function blocks
converts = {
    "i": convert_to_int,
    "s": convert_to_str,
}


def convert_str_li_to_other_li(
    str_li, pattern="i", per_letter=False, sep=" ", start_row=0, end_row=None
):
    """ Convert a list of string to a list of other types
    
    pattern: a list of types for one item. 
        'i' for int, 's' for string
        'si' means: convert the 1st item to string, the rest to integer
        If separated items are more than pattern items,
        use the last one from the parttern.
    if per_letter=True, ignore sep and separate item per letter
    """
    target_str_li = str_li[start_row:end_row]
    # find max item num
    max_item_num = 1
    if per_letter:
        max_item_num = max([len(s) for s in target_str_li])
    else:
        max_item_num = max([len(s.split(sep)) for s in target_str_li])

    # extend the pattern to the max itme num
    pattern = (
        pattern + f"{pattern[-1]}" * (max_item_num - len(pattern))
        if max_item_num > len(pattern)
        else pattern
    )

    # convert
    if per_letter:
        return [
            [converts[pattern[idx]](item) for idx, item in enumerate(s)]
            for s in target_str_li
        ]
    else:
        if sep == " ":
            return [
                [converts[pattern[idx]](item) for idx, item in enumerate(s.split())]
                for s in target_str_li
            ]
        else:
            return [
                [converts[pattern[idx]](item) for idx, item in enumerate(s.split(sep))]
                for s in target_str_li
            ]

In [3]:
fp = "input.txt"
lines = read_file_to_str_li(fp)

print("Convert to:")

# head = convert_str_li_to_other_li(
#     lines, pattern="s", per_letter=True, sep=",", start_row=0, end_row=1
# )

# print(f"Head:\n{head}")
# print(f"First line: {head[0]}")
# print(f"Last line: {head[-1]}")

data = convert_str_li_to_other_li(
    lines, pattern="s", per_letter=True, sep=",", start_row=None, end_row=None
)

print(f"First line: {data[0]}")
print(f"Last line: {data[-1]}")
print("-" * 6)

Read from input.txt:

First line: v.>vv..v>>.>>..vv......>..>.>.>.>..>v>v.......>>>.v.>>v..>.v...>>.v>.v>..>.>vvvv>
>.v..>v>v>...>.>.>.vvvv.>vv>..>v.>...>...>>vv...>.>>.v>>v. | Last line: .>...v.>v....>.>v>.v.
>>.v..v..vvv.v>>v>....vvvvv.vv>v...>v......v...>..v.>>.>..vvv...v.>..>.vv..vv..>..vv>>>>>v..v
..v>.>.v>..v..>..>..>...v

------

Convert to:

First line: ['v', '.', '>', 'v', 'v', '.', '.', 'v', '>', '>', '.', '>', '>', '.', '.', 'v', 
'v', '.', '.', '.', '.', '.', '.', '>', '.', '.', '>', '.', '>', '.', '>', '.', '>', '.', 
'.', '>', 'v', '>', 'v', '.', '.', '.', '.', '.', '.', '.', '>', '>', '>', '.', 'v', '.', 
'>', '>', 'v', '.', '.', '>', '.', 'v', '.', '.', '.', '>', '>', '.', 'v', '>', '.', 'v', 
'>', '.', '.', '>', '.', '>', 'v', 'v', 'v', 'v', '>', '>', '.', 'v', '.', '.', '>', 'v', 
'>', 'v', '>', '.', '.', '.', '>', '.', '>', '.', '>', '.', 'v', 'v', 'v', 'v', '.', '>', 
'v', 'v', '>', '.', '.', '>', 'v', '.', '>', '.', '.', '.', '>', '.', '.', '.', '>', '>', 
'v', 'v', '.', '.', '.', '>', '.', '>', '>', '.', 'v', '>', '>', 'v', '.']

Last line: ['.', '>', '.', '.', '.', 'v', '.', '>', 'v', '.', '.', '.', '.', '>', '.', '>', 
'v', '>', '.', 'v', '.', '>', '>', '.', 'v', '.', '.', 'v', '.', '.', 'v', 'v', 'v', '.', 
'v', '>', '>', 'v', '>', '.', '.', '.', '.', 'v', 'v', 'v', 'v', 'v', '.', 'v', 'v', '>', 
'v', '.', '.', '.', '>', 'v', '.', '.', '.', '.', '.', '.', 'v', '.', '.', '.', '>', '.', 
'.', 'v', '.', '>', '>', '.', '>', '.', '.', 'v', 'v', 'v', '.', '.', '.', 'v', '.', '>', 
'.', '.', '>', '.', 'v', 'v', '.', '.', 'v', 'v', '.', '.', '>', '.', '.', 'v', 'v', '>', 
'>', '>', '>', '>', 'v', '.', '.', 'v', '.', '.', 'v', '>', '.', '>', '.', 'v', '>', '.', 
'.', 'v', '.', '.', '>', '.', '.', '>', '.', '.', '>', '.', '.', '.', 'v']

------

In [5]:
sea_map = np.array(data)
sea_map = np.where(sea_map == ".", 0, sea_map)
sea_map = np.where(sea_map == ">", 1, sea_map)
sea_map = np.where(sea_map == "v", 2, sea_map)
sea_map = sea_map.astype("int")


def run_one_step(sea_map):
    for axis, cucumber_num in zip([1, 0], [1, 2]):
        move_all = np.roll(sea_map, 1, axis=axis)
        moved_mask = (move_all == cucumber_num) & (sea_map == 0)
        can_move_mask = np.roll(moved_mask, -1, axis=axis)
        moved_sea_map = np.where(moved_mask, move_all, np.NaN)
        new_empty_sea_map = np.where(can_move_mask, 0, np.NaN)
        sea_map = (
            pd.DataFrame(moved_sea_map)
            .fillna(pd.DataFrame(new_empty_sea_map))
            .fillna(pd.DataFrame(sea_map))
            .to_numpy()
        ).astype("int")
    return sea_map


count = 0
while True:
    new_sea_map = run_one_step(sea_map)
    count += 1
    if np.array_equal(sea_map, new_sea_map):
        break
    sea_map = new_sea_map
print(f"Answer to Q1: {count}")

Answer to Q1: 305